## Постановка задачи
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Разделите выборку на обучающую/проверочную в соотношении 80/20.

Постройте **ансамбль** решающих деревьев, используя **градиентный бустинг (GradientBoostingClassifier)**. Используйте перекрестную проверку, чтобы найти наилучшие параметры ансамбля, *или* используйте параметры от случайного леса: max_depth=17, max_features=27, min_samples_leaf=20, n_estimators=76.

Проведите предсказание и проверьте качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import preprocessing

### Загрузка данных

In [5]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [6]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)

data = pd.get_dummies(data=data,prefix='Product_Info_2_1')
data.fillna(value=-1, inplace=True)

print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 133 entries, Id to Product_Info_2_1_E
dtypes: float64(18), int64(110), uint8(5)
memory usage: 58.3 MB
None
   Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   2               1              10        0.076923               2   
1   5               1              26        0.076923               2   
2   6               1              26        0.076923               2   
3   7               1              10        0.487179               2   
4   8               1              26        0.230769               2   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0               1               1  0.641791  0.581818  0.148536  ...   
1               3               1  0.059701  0.600000  0.131799  ...   
2               3               1  0.029851  0.745455  0.288703  ...   
3               3               1  0.164179  0.672727  0.205021  ...   
4        

### Набор столбцов для расчета

In [7]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [8]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [9]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [10]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [11]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3         4         5         6  \
31452 -1.594727 -0.955078  0.138306 -1.542969  0.611816 -0.169434 -1.159180   
36767  0.661133  1.983398 -1.224609 -0.384277 -1.634766 -0.169434  0.862305   
4180   0.778809  0.269287 -1.073242  0.812500  0.611816 -0.169434 -1.159180   
23417 -0.208130 -0.710449 -0.694336  0.239746  0.611816 -0.169434 -1.159180   
9738   1.249023  0.514160  1.727539  1.193359 -1.634766 -0.169434  0.862305   

              7         8         9  ...       109       110       111  \
31452  1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
36767 -1.013672  0.863770 -0.928711  ... -0.083679  0.441650 -0.149292   
4180   1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
23417  1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
9738  -1.013672  0.862305  0.100891  ... -0.083679 -2.263672 -0.149292   

            112       113      114       115       116       117  Response  
314

### Градиентный бустинг
Деревья для градиентного бустинга строятся последовательно для минимизации ошибки предыдущего дерева (или деревьев). При этом в самом дереве разбиение выполняется по минимизации информационных потерь, без учета сортировки исходных данных по количеству информации в них.

In [13]:
x = pd.DataFrame(data_train, columns=columns_transformed)
model = GradientBoostingClassifier(random_state=17, max_depth=14,
                max_features=27, min_samples_leaf=20, n_estimators=76)

Диапазон тестирования параметров модели ограничен только вычислительной мощностью. Для проверки модели имеет смысл провести индивидуальные перекрестные проверки для каждого параметра в отдельности, затем в итоговой проверке перепроверить самые лучшие найденные параметры с отклонением +/-10%.

In [14]:
tree_params = {
    'max_depth': range(12,15),
   'max_features': range(25,28),
    'n_estimators': range(74,77),
    'min_samples_leaf': range(20,23)
}
tree_grid = GridSearchCV(model, tree_params, cv=5, n_jobs=4,
                         verbose=True,
                         scoring=make_scorer(cohen_kappa_score))
tree_grid.fit(x, data_train['Response'])
print (tree_grid.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


KeyboardInterrupt: 

    Fitting 5 folds for each of 81 candidates, totalling 405 fits
    [Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
    [Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 80.0min
    [Parallel(n_jobs=2)]: Done 196 tasks      | elapsed: 335.1min
    [Parallel(n_jobs=2)]: Done 405 out of 405 | elapsed: 741.1min finished
    {'max_depth': 13, 'max_features': 26, 'min_samples_leaf': 21, 'n_estimators': 75}

Выведем самые оптимальные параметры и построим итоговую модель

In [ ]:
print (tree_grid.best_params_)
model = GradientBoostingClassifier(random_state=17,
    min_samples_leaf=tree_grid.best_params_['min_samples_leaf'],
    max_depth=tree_grid.best_params_['max_depth'],
    max_features=tree_grid.best_params_['max_features'])

In [ ]:
model.fit(x, data_train['Response'])

### Предсказание данных и оценка модели

In [ ]:
x_test = pd.DataFrame(data_test, columns=columns_transformed)
data_test["target"] = model.predict(x_test)

Кластеризация дает 0.192, kNN(100) - 0.3, лог. регрессия - 0.512/0.496, SVM - 0.95, реш. дерево - 0.3, случайный лес - 0.487, XGBoost - 0.522

In [ ]:
print ("Градиентный бустинг:",
       round(cohen_kappa_score(data_test["target"],
                    data_test["Response"], weights='quadratic'), 3))

В соревновании на Kaggle 0.56 - **2010 место**

### Матрица неточностей

In [ ]:
print ("Градиентный бустинг\n",
    confusion_matrix(data_test["target"], data_test["Response"]))